In [ ]:
class ValidatedDatePicker(widgets.DatePicker):
    # הוספת שדה מצב להתעלמות זמנית מולידציה בזמן הקלדה
    _is_typing = False
    
    # הוספת תכונה לשמירת הערך הקודם
    _previous_value = Unicode('').tag(sync=True)
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._previous_value = self.value.isoformat() if self.value else ''
        
    @validate('value')
    def _valid_value(self, proposal):
        new_value = proposal['value']
        
        # אם זה null או שווה לערך הקודם, נאפשר את השינוי
        if new_value is None or (isinstance(new_value, str) and not new_value):
            return new_value
            
        try:
            # המרה לתאריך אם זה string
            if isinstance(new_value, str):
                new_value = datetime.strptime(new_value, '%Y-%m-%d').date()
            elif isinstance(new_value, datetime):
                new_value = new_value.date()
                
            # בדיקת טווח תאריכים רק אם יש תאריך תקין
            if MIN_DATE.date() <= new_value <= MAX_DATE.date():
                self._previous_value = new_value.isoformat()
                return new_value
            else:
                return datetime.strptime(self._previous_value, '%Y-%m-%d').date()
        except ValueError:
            # במקרה של שגיאה בפירוק התאריך, נחזיר את הערך הקודם
            return datetime.strptime(self._previous_value, '%Y-%m-%d').date()

# הגדרת תאריכים מינימליים ומקסימליים
MIN_DATE = datetime(2000, 1, 1)
MAX_DATE = datetime(2024, 12, 31)  # הרחבתי את הטווח לסוף 2024

def validate_dates():
    """בדיקת תקינות היחס בין התאריכים"""
    start_date = pick_start.value
    end_date = pick_end.value
    
    if start_date and end_date:
        if start_date > end_date:
            pick_start.value = end_date
        elif end_date < start_date:
            pick_end.value = start_date

# יצירת רכיבי בחירת התאריכים
pick_start = ValidatedDatePicker(
    value=datetime(2023, 1, 1).date(),
    description='Start Date',
    disabled=False
)

pick_end = ValidatedDatePicker(
    value=datetime.today().date(),
    description='End Date',
    disabled=False
)

# עדכון ערכים רק כשמסיימים להקליד
def on_value_change(change):
    if not change.owner._is_typing:
        validate_dates()

pick_start.observe(on_value_change, names='value')
pick_end.observe(on_value_change, names='value')

update_button = widgets.Button(description="Update Map")
center_layout = widgets.Layout(display='flex', align_items='center', width='100%')
widgets_HBox = widgets.HBox(children=[pick_start, pick_end, update_button], layout=center_layout)

display(widgets_HBox)